<a id = "1.0"></a>
<p style="font-size:25px;color:#34656d;font-family:'Comic Sans MS';text-align:center;border-radius:5px;">
<strong>LightGBM - Clase 5 - V1.0</strong></p> 

<a id = "1.0"></a>
<p style="font-size:25px;color:#34656d;font-family:'Comic Sans MS';text-align:center;border-radius:5px;">
<strong>______________________</strong></p> 

In [22]:
import gzip


In [4]:
# Lee el archivo como un DataFrame
data_full  = pd.read_csv('data/sell-in.csv', delimiter='\t')
df_pid_validos  = pd.read_csv('data/productos_a_predecir.txt')

# Ajustar el ancho máximo de las columnas
pd.set_option('display.max_colwidth', None)

# Ajustar el ancho máximo de la visualización
pd.set_option('display.width', 1000)

# Muestra las primeras filas del DataFrame
print(data_full.head())
data_full.info()
print(df_pid_validos.head())

#  Si tomo el archivo desde el buket de la maquina virtual del laboratorio
#
# Ruta del archivo ventas comprimido
# file_path = 'data/sell-in.txt.gz'

# Abre el archivo comprimido
#with gzip.open(file_path, 'rt') as f:
#    # Lee el archivo CSV usando pandas, especificando que el separador es un tabulador
#    data_full = pd.read_csv(f, sep='\t')



   periodo  customer_id  product_id  plan_precios_cuidados  cust_request_qty  cust_request_tn       tn
0   201701        10234       20524                      0                 2          0.05300  0.05300
1   201701        10032       20524                      0                 1          0.13628  0.13628
2   201701        10217       20524                      0                 1          0.03028  0.03028
3   201701        10125       20524                      0                 1          0.02271  0.02271
4   201701        10012       20524                      0                11          1.54452  1.54452
   product_id
0       20001
1       20002
2       20003
3       20004
4       20005


In [5]:
# Armado de la lista de productos validos a predecir para el periodo
product_ids_validos = df_pid_validos['product_id'].tolist()

# Ver cuántos registros tiene la lista
num_registros = len(product_ids_validos)
print(f"La lista product_ids_validos tiene {num_registros} registros.")

La lista product_ids_validos tiene 780 registros.


In [24]:
# Filtrar el DataFrame original para quedarse solo con los product_id válidos
data = data_full[data_full['product_id'].isin(product_ids_validos)]

# Ver cuántos registros tiene el DataFrame filtrado
num_registros_filtrados = len(data)
print(f"El DataFrame filtrado tiene {num_registros_filtrados} registros.")

# Verificar que todos los product_id en el DataFrame filtrado están en la lista de productos válidos
productos_unicos_filtrados = data['product_id'].unique()
productos_invalidos = [pid for pid in productos_unicos_filtrados if pid not in product_ids_validos]

if len(productos_invalidos) == 0:
    print("Todos los product_id en el DataFrame filtrado son válidos.")
else:
    print(f"Se encontraron productos no válidos en el DataFrame filtrado: {productos_invalidos}")

# (Opcional) Ver cuántos registros hay por cada product_id
registros_por_producto = data['product_id'].value_counts()
print("Número de registros por product_id en el DataFrame filtrado:")
print(registros_por_producto)

# Ver como esta el data frame
print(data.head())
data.info()

El DataFrame filtrado tiene 2293481 registros.
Todos los product_id en el DataFrame filtrado son válidos.
Número de registros por product_id en el DataFrame filtrado:
product_id
20111    7973
20122    7950
20120    7537
20326    7397
20132    7199
         ... 
21267      67
21252      67
21276      64
20886      63
20953      62
Name: count, Length: 780, dtype: int64
   periodo  customer_id  product_id  plan_precios_cuidados  cust_request_qty  cust_request_tn       tn
0   201701        10234       20524                      0                 2          0.05300  0.05300
1   201701        10032       20524                      0                 1          0.13628  0.13628
2   201701        10217       20524                      0                 1          0.03028  0.03028
3   201701        10125       20524                      0                 1          0.02271  0.02271
4   201701        10012       20524                      0                11          1.54452  1.54452
<class 'pan

In [15]:
# Etapa 1: Preparación de los Datos

In [26]:
# Filtrar los datos para el período especificado
#data['periodo'] = data['periodo'].astype(str)
#data = data[(data['periodo'] >= '201801') & (data['periodo'] <= '201812')]

# Agrupar por product_id
#products = data['product_id'].unique()

import pandas as pd

# Verificar el tipo de dato de la columna 'periodo'
print(data['periodo'].dtype)

# Obtener los 10 product_id con más ventas en todo el dataframe
top_products = data.groupby('product_id')['tn'].sum().nlargest(10).index

# Filtrar el dataframe para quedarse solo con estos productos
filtered_data = data[data['product_id'].isin(top_products)].copy()

# Convertir 'periodo' a un formato datetime
filtered_data.loc[:, 'periodo'] = pd.to_datetime(filtered_data['periodo'].astype(str) + '01', format='%Y%m%d')

# Crear características temporales adicionales si es necesario
filtered_data.loc[:, 'month'] = filtered_data['periodo'].dt.month
filtered_data.loc[:, 'year'] = filtered_data['periodo'].dt.year

int64


C:\Users\hbasilio\AppData\Local\Temp\ipykernel_23840\649171342.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '<DatetimeArray>
['2017-01-01 00:00:00', '2017-01-01 00:00:00', '2017-01-01 00:00:00', '2017-01-01 00:00:00', '2017-01-01 00:00:00', '2017-01-01 00:00:00', '2017-01-01 00:00:00', '2017-01-01 00:00:00', '2017-01-01 00:00:00', '2017-01-01 00:00:00',
 ...
 '2019-12-01 00:00:00', '2019-12-01 00:00:00', '2019-12-01 00:00:00', '2019-12-01 00:00:00', '2019-12-01 00:00:00', '2019-12-01 00:00:00', '2019-12-01 00:00:00', '2019-12-01 00:00:00', '2019-12-01 00:00:00', '2019-12-01 00:00:00']
Length: 62257, dtype: datetime64[ns]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  filtered_data.loc[:, 'periodo'] = pd.to_datetime(filtered_data['periodo'].astype(str) + '01', format='%Y%m%d')


In [27]:
# Verificar los datos filtrados
print(filtered_data.head())
filtered_data.info()

        periodo  customer_id  product_id  plan_precios_cuidados  cust_request_qty  cust_request_tn       tn  month  year
4766 2017-01-01        10063       20007                      0                 1          0.81730  0.81730      1  2017
4767 2017-01-01        10046       20007                      0                 1          8.17296  8.17296      1  2017
4768 2017-01-01        10027       20007                      0                 2          0.38919  0.38919      1  2017
4769 2017-01-01        10045       20007                      0                 1          1.63460  1.63460      1  2017
4770 2017-01-01        10488       20007                      0                 1          0.24324  0.24324      1  2017
<class 'pandas.core.frame.DataFrame'>
Index: 62257 entries, 4766 to 2916470
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   periodo                62257 non-null  date

In [28]:
# Etapa 2: Entrenamiento del Modelo LightGBM

In [37]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split

# Crear las características y la variable objetivo
X = filtered_data.drop(columns=['tn', 'periodo', 'customer_id'])
y = filtered_data['tn']

# Dividir los datos en conjuntos de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear los datasets de LightGBM
train_data = lgb.Dataset(X_train, label=y_train)
val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)

# Definir los parámetros del modelo
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'learning_rate': 0.05,
    'num_leaves': 31,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1
}

# Entrenar el modelo con devoluciones de llamada
model = lgb.train(
    params,
    train_data,
    valid_sets=[train_data, val_data],
    num_boost_round=1000,
    callbacks=[
        lgb.early_stopping(stopping_rounds=50),
        lgb.log_evaluation(period=100)  # Controla la frecuencia de la salida detallada
    ]
)

# Verificar el modelo
print('Entrenamiento completado')

Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 3.25883	valid_1's rmse: 3.29998
Early stopping, best iteration is:
[65]	training's rmse: 3.56097	valid_1's rmse: 3.2663
Entrenamiento completado


In [38]:
# Etapa 3: Predicción para Febrero de 2020

In [42]:
import pandas as pd

# Crear el dataframe de características agregadas por producto y mes
agg_features = filtered_data.groupby(['product_id', 'month', 'year']).agg({
    'plan_precios_cuidados': 'mean',
    'cust_request_qty': 'mean',
    'cust_request_tn': 'mean'
}).reset_index()

# Seleccionar las características agregadas para el último mes disponible (diciembre de 2019)
dec_2019_features = agg_features[(agg_features['month'] == 12) & (agg_features['year'] == 2019)].copy()

# Ajustar los datos para febrero de 2020
feb_2020_df = dec_2019_features.copy()
feb_2020_df['month'] = 2
feb_2020_df['year'] = 2020

# Seleccionar solo los top_products para la predicción
feb_2020_df = feb_2020_df[feb_2020_df['product_id'].isin(top_products)].copy()

# Asegurarse de que el DataFrame tenga las mismas columnas que las usadas en el entrenamiento
features_used = ['plan_precios_cuidados', 'cust_request_qty', 'cust_request_tn', 'month', 'year']
feb_2020_df = feb_2020_df[features_used]

# Realizar las predicciones
predictions = model.predict(feb_2020_df)

# Crear el dataframe final con las predicciones
result_df = pd.DataFrame({
    'product_id': feb_2020_df['product_id'],
    'tn': predictions
})

# Guardar las predicciones en un archivo CSV
result_df.to_csv('predicciones_febrero_2020.csv', index=False)

# Verificar el resultado
print(result_df)

LightGBMError: The number of features in data (5) is not the same as it was in training data (6).
You can set ``predict_disable_shape_check=true`` to discard this error, but please be aware what you are doing.

<a id = "1.0"></a>
<p style="font-size:25px;color:#34656d;font-family:'Comic Sans MS';text-align:center;border-radius:5px;">
<strong>______________________</strong></p> 